In [89]:
%matplotlib notebook
import time
import numpy as np
from matplotlib import rcParams
from matplotlib.animation import PillowWriter
from matplotlib.animation import FFMpegWriter
from IPython.display import Image, Video
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm import tqdm


In [90]:
rcParams['animation.embed_limit'] = 40
m = 1
eps = 1
sig = 1
Lx, Ly, Lz = (10, 10, 10)
dimensions = [Lx, Ly, Lz]
grid_size = 2.5*sig
N = 100

display_interval = 100
dt = 1e-3

sqrt_threshold = 0

In [91]:
def lennard_jones_potential(r, dir):
    term1 = (sig / r) ** 12
    term2 = (sig / r) ** 6
    f = (((24 * eps)/r) * (2* term1 - term2))*dir
    return f

In [92]:
def lennard_jones_potential_sqrd(r_sqrd, dir):
    term1 = 24/r_sqrd**8
    term2 = (2/r_sqrd**6)-1
    f = (term1 * term2)*dir
    return f

In [93]:
def is_not_adjacent(g1, g2):
    g_d = np.abs(g1 - g2)
    for d in g_d:
        if d > 1 and d < Lx/grid_size -1:
            return True
    
    return False

In [94]:
def get_grid(xyz):
    grid = []
    for ax in xyz:
        #introduce small shift to avoid wrong classification at the border
        grid.append(np.floor(ax / (grid_size + 0.0000001)))
    return np.array(grid)

In [95]:
class Point :
    def __init__ (self, x, y, z):
        self.x = np.array([x, y, z])
        self.v = np.array([np.random.uniform(0, 0.1),
                            np.random.uniform(0, 0.1),
                            np.random.uniform(0, 0.1)])
        
        self.f = np.zeros(3)
        self.grid = get_grid([x,y,z])
        #verlet
        self.last_x = np.array([x, y, z])
        
    def calculate_force(self, p2):
        dir = (self.x - p2.x) / np.linalg.norm(self.x - p2.x)
        dx = self.x[0] - p2.x[0]
        dy = self.x[1] - p2.x[1]
        dz = self.x[2] - p2.x[2]
        dx = dx - np.rint(dx/Lx) * Lx
        dy = dy - np.rint(dy/Ly) * Ly
        dz = dz - np.rint(dz/Lz) * Lz
        r = np.sqrt(dx**2 + dy**2 + dz**2)
        f = lennard_jones_potential(r, dir)
        self.f += f
        p2.f -= f
    def calculate_force_nosqrt( self, p2):
        dir = (self.x - p2.x)
        dx = self.x[0] - p2.x[0]
        dy = self.x[1] - p2.x[1]
        dz = self.x[2] - p2.x[2]
        dx = dx - np.rint(dx/Lx) * Lx
        dy = dy - np.rint(dy/Ly) * Ly
        dz = dz - np.rint(dz/Lz) * Lz
        r_sqrd = dx**2 + dy**2 + dz**2
        f = lennard_jones_potential_sqrd(r_sqrd, dir)
        self.f += f
        p2.f -= f
    def update_velocity(self):
        self.v += (dt/m) * self.f
        self.f = np.zeros(3)
    def update_position(self):
        #semi-implicid euler
        self.x = self.x + self.v * dt
        
        #velocity-verlet
        
        #strömer-verlet
        # last = self.x
        # self.x = 2*self.x - self.last_x + self.v * dt
        # self.last_x = last
        for i in range(3):
            if self.x[i] < 0:
                self.x[i] += dimensions[i]
            elif self.x[i] > dimensions[i]:
                self.x[i] -= dimensions[i]
        self.grid = get_grid(self.x)

In [96]:
points = [Point(np.random.uniform(0, Lx),
                np.random.uniform(0, Ly),
                np.random.uniform(0, Lz))
          for _ in range(N)]
repositioning = True

while repositioning:
    index = 0
    for i, p1 in enumerate(points):
            for ii, p2 in enumerate(points):
                if ii <= i:
                    continue
                else:
                    dx = p1.x[0] - p2.x[0]
                    dy = p1.x[1] - p2.x[1]
                    dz = p1.x[2] - p2.x[2]
                    dx = dx - np.rint(dx/Lx) * Lx
                    dy = dy - np.rint(dy/Ly) * Ly
                    dz = dz - np.rint(dz/Lz) * Lz
                    r = np.sqrt(dx**2 + dy**2 + dz**2)
                    if r < 1.12*sig:
                        index += 1
                        p1.x = np.array([np.random.uniform(0, Lx),
                                np.random.uniform(0, Ly),
                                np.random.uniform(0, Lz)])
    if index == 0:
         repositioning = False

In [97]:
def update_positions_and_velocities(points, sqrd=True):
    if sqrd == True:
        for i, p1 in enumerate(points):
            for ii, p2 in enumerate(points):
                if ii <= i or is_not_adjacent(p1.grid, p2.grid):
                    continue
                else:
                    p1.calculate_force(p2)
    elif sqrd == False:
        for i, p1 in enumerate(points):
            for ii, p2 in enumerate(points):
                if ii <= i or is_not_adjacent(p1.grid, p2.grid):
                    continue
                else:
                    p1.calculate_force_nosqrt(p2)
    
    for p in points:
        p.update_velocity()
        p.update_position()

In [98]:
def get_temp():
    K = 0
    for p in points:
        K += np.linalg.norm(p.v)**2
    
    return  K/(3*(N-1))

In [99]:
def get_animation(points):
    num_frames = 50 # Number of frames for animation
    fig = plt.figure(figsize=(12, 6))
    progress_bar = tqdm(total=num_frames, desc="Rendering frames", ncols=80)

    #3D plot
    ax = fig.add_subplot(121, projection='3d')
    ax.set_xlim(0, Lx)
    ax.set_ylim(0, Ly)
    ax.set_zlim(0, Lz)

    # Line plot
    ax2 = fig.add_subplot(122)
    ax2.set_xlim(0, num_frames)
    ax2.set_ylim(0, 1)
    line, = ax2.plot([], [], lw=2)

    points_draw = [ax.plot([], [], [], 'o', markersize = 5, color = "teal")[0] for _ in range(N)]

    # Add a textbox to display the steps per second
    step_counter_text = ax.text2D(0.05, 0.95, '', transform=ax.transAxes)

    #init line data
    line_data = []

    def init():
        for point in points_draw:
            point.set_data([], [])
            point.set_3d_properties([])
        line.set_data([], [])
        step_counter_text.set_text('')
        return points_draw  + [step_counter_text, line]

    def update(frame):
        start_time = time.time()  
        if frame < sqrt_threshold*num_frames:
            for i in range(display_interval):
                update_positions_and_velocities(points)
        else:
            for i in range(display_interval):
                update_positions_and_velocities(points, sqrd=False)
        end_time = time.time()
        steps_per_second = N * display_interval / (end_time + 0.00001 - start_time)
        for p, p_d in zip(points, points_draw):
            p_x = p.x
            p_d.set_data_3d([p_x[0], p_x[1], p_x[2]])
        
        step_counter_text.set_text(f'Steps/s: {round(steps_per_second / 100) * 100:.2f}')

        line_data.append(get_temp())
        line.set_data(range(len(line_data)), line_data)

        progress_bar.update(1)
        return points_draw + [step_counter_text, line]
    
    ani = FuncAnimation(fig, update, frames=num_frames, init_func=init, blit=True, interval = dt*1e4*display_interval)
    return ani, progress_bar


# Call the get_animation function to create the animation object
ani, progress_bar = get_animation(points)
writer = FFMpegWriter(fps = 30)
ani.save('animation.mp4', writer=writer, dpi=80)

progress_bar.close()

#Image(filename="animation.gif")
Video("animation.mp4")
# Display the animation as an HTML video
#HTML(ani.to_jshtml())


<IPython.core.display.Javascript object>

Rendering frames: 100%|█████████████████████████| 50/50 [05:11<00:00,  6.23s/it]
